# Exercise 3a

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Load dataset

In [ ]:
trainset = torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

### Model

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 4 * 4, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = self.fc2(x)
    def eval(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        #x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        #x = self.fc2(x)

        return x

model = Net()

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.000001)

writer = SummaryWriter()

### Training

In [ ]:
best_accuracy = 0
best_net = 0

for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train 1", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels

        outputs = model(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train 1", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [12000/12000] - Loss: 0.20127514004707336, accuracy:  0.9693333333333334
 (new best)
Epoch 2 [12000/12000] - Loss: 0.06821391731500626, accuracy:  0.979
 (new best)
Epoch 3 [12000/12000] - Loss: 0.05175609141588211, accuracy:  0.9828333333333333
 (new best)
Epoch 4 [12000/12000] - Loss: 0.0037986994720995426, accuracy:  0.9835833333333334
 (new best)
Epoch 5 [12000/12000] - Loss: 0.0045074401423335075, accuracy:  0.9814166666666667
Finished Training


### Test

In [ ]:
correct = 0
total = 0
for i, data in enumerate(test_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.9851
